In [1]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
import os
from tqdm.notebook import tqdm

In [ ]:
%%html
<style>
.cell-output-ipywidget-background {
    background-color: transparent !important;
}
:root {
    --jp-widgets-color: var(--vscode-editor-foreground);
    --jp-widgets-font-size: var(--vscode-editor-font-size);
}  
</style>

In [3]:
spark = (
    SparkSession.builder
        .appName("TestTransformFile")
        .config("spark.sql.shuffle.partitions", "4")
        .getOrCreate()
    )

In [15]:
# test enums in python

from enum import Enum

# compression options enum:
class CompressionOptions(Enum):
    ZLIB = "zlib"
    GZIP = "gzip"
    NONE = None

test_val_1 = CompressionOptions.NONE
test_val_2 = CompressionOptions.ZLIB

if (CompressionOptions.NONE == test_val_1):
    print("0") # prints
if (CompressionOptions.ZLIB == test_val_1):
    print("1")
if (CompressionOptions.NONE == test_val_2):
    print("2")
if (CompressionOptions.ZLIB == test_val_2):
    print("3") # prints

0
3


In [16]:
def convert_csv_to_other_format(csv_file_name, output=True, format="parquet", compression=CompressionOptions.ZLIB):
    with tqdm(total=3) as pbar:
        csv_path = f"./work/data/{csv_file_name}"
        other_file_name = f"{csv_file_name.split('.')[0]}.{format}"
        other_path = f"./work/data/{other_file_name}"
        pbar.update()
        
        if output:
            print(
                csv_file_name,
                round(os.path.getsize(csv_path) / 1024 /1024, 2),
                "MB"
            )
        
        data_csv = spark.read.csv(csv_path, header=True, inferSchema=True)
        pbar.update()
        write_builder = data_csv.write.format(format)
        if compression != CompressionOptions.NONE:
            write_builder.option("compression", compression.value)
        write_builder.save(other_path, mode='overwrite')
        pbar.update()
        
        if output:
            print(
                other_file_name,
                round(
                    sum(
                        os.path.getsize(f"{other_path}/{f}") 
                        for f in os.listdir(f"{other_path}")
                        if os.path.isfile(f"{other_path}/{f}")
                    ) / 1024 / 1024,
                    2
                ),
                "MB"
            )

Parquet format

doesn't support zlib compression

In [22]:
convert_csv_to_other_format("customers-100000.csv", output=True, format="parquet", compression=CompressionOptions.GZIP)

  0%|          | 0/3 [00:00<?, ?it/s]

customers-100000.csv 17.17 MB
customers-100000.parquet 6.0 MB


In [23]:
convert_csv_to_other_format("customers-100000.csv", output=True, format="parquet", compression=CompressionOptions.NONE)

  0%|          | 0/3 [00:00<?, ?it/s]

customers-100000.csv 17.17 MB
customers-100000.parquet 9.29 MB


In [24]:
convert_csv_to_other_format("orders-from-customers-100000.csv", output=True, format="parquet", compression=CompressionOptions.GZIP)

  0%|          | 0/3 [00:00<?, ?it/s]

orders-from-customers-100000.csv 29.23 MB
orders-from-customers-100000.parquet 10.99 MB


In [25]:
convert_csv_to_other_format("orders-from-customers-100000.csv", output=True, format="parquet", compression=CompressionOptions.NONE)

  0%|          | 0/3 [00:00<?, ?it/s]

orders-from-customers-100000.csv 29.23 MB
orders-from-customers-100000.parquet 17.14 MB


ORC format

doesn't support Gzip compression

In [26]:
convert_csv_to_other_format("customers-100000.csv", output=True, format="orc", compression=CompressionOptions.ZLIB)

  0%|          | 0/3 [00:00<?, ?it/s]

customers-100000.csv 17.17 MB
customers-100000.orc 5.52 MB


In [27]:
convert_csv_to_other_format("customers-100000.csv", output=True, format="orc", compression=CompressionOptions.NONE)

  0%|          | 0/3 [00:00<?, ?it/s]

customers-100000.csv 17.17 MB
customers-100000.orc 5.23 MB


In [28]:
convert_csv_to_other_format("orders-from-customers-100000.csv", output=True, format="orc", compression=CompressionOptions.ZLIB)

  0%|          | 0/3 [00:00<?, ?it/s]

orders-from-customers-100000.csv 29.23 MB
orders-from-customers-100000.orc 10.78 MB


In [29]:
convert_csv_to_other_format("orders-from-customers-100000.csv", output=True, format="orc", compression=CompressionOptions.NONE)

  0%|          | 0/3 [00:00<?, ?it/s]

orders-from-customers-100000.csv 29.23 MB
orders-from-customers-100000.orc 10.25 MB
